# Extract Detected Changes in Sentinel-1 Imagery

Inspired by the following Google Earth Engine tutorials: 
- [Part 2 - Hypothesis testing](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2); and
- [Part 3 - Multitemporal change detection](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3)

## Part 2 - Detecting Changes in Sentinel-1 Imagery

In [1]:
# Run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import pandas as pd
import IPython.display as disp
import json
import csv 
import os
import datetime
import requests
from datetime import datetime
from datetime import timedelta
import time
%matplotlib inline

geoJSON_path = "C:\\Desktop\\AI4ER\\03 - MRes\\Easter 2023\\MRes Project\\OneDrive - University of Cambridge\\geoJSON"
output_path = "r'C:\\Desktop\\AI4ER\\03 - MRes\\Easter 2023\\MRes Project\\OneDrive - University of Cambridge\\satelliteSIGNALS"
date_info = pd.read_excel(
    'C:\\Desktop\\AI4ER\\03 - MRes\\Easter 2023\\MRes Project\\ArcticCCAM\\data\\landslide_incidents\\landslide_incidents_27341_since20150623_alongroad_norwayonly.xlsx')

In [3]:
#create function to download the image or imagecollection as you desire
def downloader(ee_object,region): 
    try:
        #download image
        if isinstance(ee_object, ee.image.Image):
            print('Its Image')
            url = ee_object.getDownloadUrl({
                    'scale': 10,
                    'crs': 'EPSG:4326',
                    'region': region
                })
            return url
        
        #download imagecollection
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            print('Its ImageCollection')
            ee_object_new = ee_object.mosaic()
            url = ee_object_new.getDownloadUrl({
                    'scale': 10,
                    'crs': 'EPSG:4326',
                    'region': region
                })
            return url
    except:
        print("Could not download")

In [4]:
def getDATES(aoi, date_event):
    
    n = 175
    output = []
    for i in range(1, n+1):
        output.append(i)
    multiple = 120 #1/3 of a year
    
    for iend in range(1,100):
        # print(iend*multiple)
        im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                    .filterBounds(aoi)
                    .filterDate(ee.Date(date_event+timedelta(days=1)),ee.Date(date_event+timedelta(days=iend*multiple)))
                    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                    .filter(ee.Filter.inList('relativeOrbitNumber_start', output))
                    .sort('system:time_start'))
        acq_times_end = im_coll.aggregate_array('system:time_start').getInfo()
        orbitN = im_coll.aggregate_array('relativeOrbitNumber_start').getInfo()
        if len(acq_times_end) > 1:
            endDATE = datetime.fromtimestamp(mktime(time.gmtime(acq_times_end[0]/1000)))
            orbitchosen = orbitN[0]
            break
    
    for istart in range(1,100):
        # print(istart*multiple)
        im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                    .filterBounds(aoi)
                    .filterDate(ee.Date(date_event-timedelta(days=istart*multiple)),ee.Date(date_event-timedelta(days=1)))
                    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                    .filter(ee.Filter.eq('relativeOrbitNumber_start', orbitchosen))
                    .sort('system:time_start'))
        acq_times_start = im_coll.aggregate_array('system:time_start').getInfo()
        if len(acq_times_start) > 1:
            startDATE = datetime.fromtimestamp(mktime(time.gmtime(acq_times_start[-1]/1000)))
            break

    return startDATE, endDATE, orbitchosen

In [5]:
def download_url(path, newfile):
    t0 = time.time()
    url = path
    fn = newfile
    try:
        r = requests.get(url)
        with open(fn, 'wb') as f:
            f.write(r.content)
        return(url, time.time() - t0)
    except Exception as e:
        print('Exception in download_url():', e)

In [9]:
# sample only
df_sample = pd.read_excel('C:\\Desktop\\AI4ER\\03 - MRes\\Easter 2023\\MRes Project\\ArcticCCAM\\data\\subset_sample_5percent_fid.xlsx')
df_sample_fid = df_sample["fid"]
df_sample_fid.values

array([ 9322, 25260,  4816, 24653, 27195,  5983, 15598,  1643, 24954,
       14262, 26556,  3567, 13617, 20889, 15382, 17200, 27028,  4424,
       19036, 25120, 14349, 16410, 11489, 14236,  5217, 11639, 12399,
       21186, 10236, 14323,  9627, 23722,  8140,  9456, 13104, 10097,
         102,  9453, 19128, 21451, 21149, 15179, 22177, 27161, 21479,
       20823, 15878,  6873,    86, 17951,  9386,  7502,  5820, 26139,
         610,  1130, 24870,  9859, 16595, 12719, 16189, 18110,  2403,
       17538,  3025, 21244,  6457, 16424,  5778,  5887, 16573,  3656,
       26580,  5014,  4212,  9956,  4660,   730,  1895,  5539,  9382,
       16047, 20937,  9946, 25383, 10389,  9178, 11510,   934, 20786,
        6560,  7490, 22376, 12288,  2260, 25769, 25268, 24538, 26754,
       20044,   110, 12246,  9836, 16767, 11201,  3853, 19025,  4437,
        4994, 10783,  5398,  8073,  9112,  8641,  7473,  6164,  7994,
       24261, 15322, 20863, 25845,  9931, 13646, 19383, 10357,  2310,
       25583, 13781,

In [10]:
t0 = time.time()
from datetime import datetime
from time import mktime
for filename in os.listdir(geoJSON_path):
    json_fid = int(filename[:-9])
    if json_fid in df_sample_fid.values:
        try:
            print(filename)
            # load geoJSON file
            with open(geoJSON_path+'\\'+filename) as fa:
                geoJSON = json.load(fa)
                
            coords = geoJSON['features'][0]['geometry']['coordinates']
            aoi = ee.Geometry.Polygon(coords)
            
        
            idx = int(np.where(date_info["fid2"] == json_fid)[0])
            date_event = datetime(date_info["YYYY"][idx], date_info["MM"][idx], date_info["DD"][idx])
            startDATE, endDATE, orbitchosen = getDATES(aoi, date_event)
            
            im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                        .filterBounds(aoi)
                        .filterDate(ee.Date(startDATE),ee.Date(endDATE+timedelta(days=1)))
                        .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                        .filter(ee.Filter.eq('relativeOrbitNumber_start', orbitchosen))
                        .sort('system:time_start')) 
            
            im_list = im_coll.toList(im_coll.size())
            acq_times = im_coll.aggregate_array('system:time_start').getInfo()
            
            im1 = ee.Image(im_list.get(0)).select('VV').clip(aoi)
            im2 = ee.Image(im_list.get(-1)).select('VV').clip(aoi)
            ratio = im1.divide(im2)

            region = aoi.toGeoJSONString()#region must in JSON format
            path = downloader(ratio,region)#call function
            newfile = str('ratio_fid2_'+str(json_fid)+'_'+'orbit'+str(orbitchosen)+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[0]/1000))).replace("/","")+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[-1]/1000))).replace("/","")+".zip")
            t0 = time.time()
            result = download_url(path, newfile)
            print('url:', result[0], 'time:', result[1])
            
            path = downloader(im1,region)#call function
            newfile = str('im1_fid2_'+str(json_fid)+'_'+'orbit'+str(orbitchosen)+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[0]/1000))).replace("/","")+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[-1]/1000))).replace("/","")+".zip")
            t0 = time.time()
            result = download_url(path, newfile)
            print('url:', result[0], 'time:', result[1])
            
            path = downloader(im2,region)#call function
            newfile = str('im2_fid2_'+str(json_fid)+'_'+'orbit'+str(orbitchosen)+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[0]/1000))).replace("/","")+'_'+
                        (time.strftime('%x', time.gmtime(acq_times[-1]/1000))).replace("/","")+".zip")
            t0 = time.time()
            result = download_url(path, newfile)
            print('url:', result[0], 'time:', result[1])
        except:
            pass
print('Total time:', time.time() - t0)

10092_gee.json
Its Image
url: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a3f5ab8d2403ac7b46ad8f6c572dff73-cb0f8a349f8ece7f0d9229af08bff4ba:getPixels time: 1.1278512477874756
Its Image
url: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/54c954159ded7cbaef89d10b66b3897b-26ef981d5b02070db41b0c4399aaee31:getPixels time: 1.1853294372558594
Its Image
url: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/d8e0b0a48200f931b4424e3cb4dfddbd-a3d996615e817a2afadd6117f5b90981:getPixels time: 0.5971717834472656
10097_gee.json
Its Image
url: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/f9354957ac74760127bc41a3031ba54c-fcc0b1e2352c9dcfbe217d31e3e860c5:getPixels time: 1.1479206085205078
Its Image
url: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/195e09b5fbac3e3b2bb86d5c51093ed6-dcc7c3b8dfa8d3b928b395c062ce631f:getPixels time: